	
Яндекс.Еда осуществляет доставку еды из ресторанов. При этом у каждого ресторана есть зона, в рамках которой осуществляется доставка. Зона представляет собой полигон (заданы координаты его вершин). Пользователь в зависимости от своего местоположения (координат точки) видит разное количество доступных ресторанов. Нам важно, чтобы у каждого пользователя было достаточное количество ресторанов для выбора. Задача заключается в том, чтобы для каждого пользователя посчитать доступное ему количество ресторанов.

Использовать Python (результат .py или .ipynb файл).

Данные, которые есть (для простоты в формате .csv, несколько первых строк): user_coordinates.csv (примерно 300 тыс. строк, user_id – идентификатор пользователя)

user_id,loc_lat,loc_lon

1,55.737564,37.345186
2,56.234564,37.234590
3,55.234578,36.295745

place_zone_coordinates.csv (примерно 500 тыс. строк,
place_id – идентификатор ресторана, point_number – порядковый номер вершины полигона)

place_id,loc_lat,loc_lon,point_number

1,55.747022,37.787073,0
1,55.751713,37.784328,1
1,55.753878,37.777638,2
1,55.751031,37.779351,3
2,55.803885,37.458311,0
2,55.808677,37.464054,1
2,55.809763,37.461314,2
2,55.810840,37.458654,3

Формат результата:

id,number_of_places_available
1,2
2,19
3,0

Пожалуйста, укажите ссылку на файл с выполненным тестовым заданием.

In [74]:
# импорт модулей
import pandas as pd
# чтение данных из файлов
df_u = pd.read_csv('user_coordinates.csv')
df_p = pd.read_csv('place_zone_coordinates.csv')

In [23]:
df_p.head()

,place_id,loc_lat,loc_lon,point_number
0,1,55.747022,37.787073,0
1,1,55.751713,37.784328,1
2,1,55.753878,37.777638,2
3,1,55.751031,37.779351,3
4,2,55.803885,37.458311,0


In [24]:
df_u.head()

,user_id,loc_lat,loc_lon
0,1,55.737564,37.345186
1,2,56.234564,37.234590
2,3,55.234578,36.295745


In [88]:
def area_triangle (a, b, c):
    """Функция определения площади треугольника по координатам вершин треугольника.
    a, b, c - координаты вершин треугольника a=[x1, y1], b=[x2, y2], c=[x3, y3].
    """
    x1=a[0]
    y1=a[1]
    x2=b[0]
    y2=b[1]
    x3=c[0]
    y3=c[1]
    sq=0.5*abs((x2-x1)*(y3-y1)-(x3-x1)*(y2-y1))
    return sq
# Проверка
area_triangle((0, 3), (0, 0), (2, 0))

3.0

In [89]:
# Для использования функции вычисления площади по координатам необходимо сферическу (долготу и широту) систему координат
# преобразовать в прямоугольную систему координат.
import numpy as np

def geotocoor (loc):
    """Преобразование широты и долготы в прямоугольную систему координат.
    loc=[latitude, longitude] в формате градусы пример [55.737564, 37.345186]"""
    at=np.radians(loc[0])  #latitude в радианах 
    on=np.radians(loc[1])  #longitude в радианах
    a=6378.1   #Экваториальный радиус, км
    b=6356.8   #Полярный радиус, км
    n=(a**2)/(((a**2)*(np.cos(at)**2)+(b**2)*(np.sin(at)**2))**0.5) #радиус кривизны первого вертикала
    x=n*np.cos(at)*np.cos(on)
    y=n*np.cos(at)*np.sin(on)
    return [x, y]

# Проверка
geotocoor(usertopoint(0))
# at=55.737564
# np.radians(57.29577)
# np.cos(np.radians(at))
# np.cos(np.pi)
# placetopoint(0)[1]

[2861.1687008828185, 2183.194534569534]

In [92]:
# Определим положение пользователей в прямоугольной системе координат и добавим в таблицу
from tqdm import tnrange, tqdm_notebook, tqdm

for i in tnrange(df_u.shape[0]):
    loc=df_u.iloc[i,1:3].values #широта и долгота
    df_u.loc[i,'X']=geotocoor(loc)[0]
    df_u.loc[i,'Y']=geotocoor(loc)[1]
df_u.head()


A Jupyter Widget

,user_id,loc_lat,loc_lon,X,Y
0,1,55.737564,37.345186,2861.168701,2183.194535
1,2,56.234564,37.234590,2828.858799,2149.913683
2,3,55.234578,36.295745,2937.863397,2157.739326


In [93]:
# Определим положение точек зон доставки в прямоугольной системе координат и добавим в таблицу
for i in tnrange(df_p.shape[0]):
    loc=df_p.iloc[i,1:3].values #широта и долгота
    df_p.loc[i,'X']=geotocoor(loc)[0]
    df_p.loc[i,'Y']=geotocoor(loc)[1]
df_p.head()


A Jupyter Widget

,place_id,loc_lat,loc_lon,point_number,X,Y
0,1,55.747022,37.787073,0,2843.558323,2204.662536
1,1,55.751713,37.784328,1,2843.322755,2204.261797
2,1,55.753878,37.777638,2,2843.422624,2203.807728
3,1,55.751031,37.779351,3,2843.563826,2204.053255
4,2,55.803885,37.458311,0,2852.006477,2185.126401


In [95]:
# создаем список id ресторанов.
places=df_p['place_id'].unique() # set(df_p['place_id']
places

# список id point_number
# df_p[df_p['place_id']==places[1]][df_p.columns[-1:]].values

array([1, 2], dtype=int64)

In [99]:
def placetopoint(i):
    """Функция возвращает координаты точек многоугольника зоны доставки ресторана по индексу списка places
    places - список id ресторанов (place_id) 
    """
    return df_p[df_p['place_id']==places[i]][df_p.columns[4:]].values

#проверка 
placetopoint(0)

array([[2843.55832281, 2204.66253632],
       [2843.32275487, 2204.26179746],
       [2843.42262389, 2203.80772805],
       [2843.56382557, 2204.05325521]])

In [132]:
def sum_area_triangles(point, listpoint):
    """Функция возвращает сумму площадей треугольников с одной общей вершиной"""
    n=len(listpoint)-1 # количество треугольноиков
    areas=0
    for i in range(n):
        areas=areas+area_triangle(point, listpoint[i], listpoint[i+1])
    return areas

# Проверка: 
# Определение площади зоны доставки ресторана с индексом 1
sum_area_triangles(placetopoint(1)[0], placetopoint(1)[1:])

0.11039238480371821

In [110]:
# создаем список id users.
users=df_u['user_id']
users

0    1
1    2
2    3
Name: user_id, dtype: int64

In [134]:
def usertopoint(i):
    """Функция возвращает координаты пользователя по индексу списка users
    users - список id пользователей (id users)"""
    return df_u[df_u['user_id']==users[i]][df_u.columns[3:]].values[0]

# Проверка: 
# координаты пользователя 
usertopoint(0)

array([2861.16870088, 2183.19453457])

In [136]:
# id пользователя
users[0]

1

In [114]:
# Площадь суммы треугольников на которые делит точнка (координаты) пользователя многоугольник зоны доставки.
sum_area_triangles(usertopoint(0), placetopoint(0))

12.690714318639323

In [115]:
# %debug
# Площадь самой зоны доставки
sum_area_triangles(placetopoint(0)[0], placetopoint(0)[1:])

0.11718409611987612

In [116]:
# Сравнение 
sum_area_triangles(usertopoint(0), placetopoint(0))==sum_area_triangles(placetopoint(0)[0], placetopoint(0)[1:])

False

In [137]:
def countplaces (user):
    """Функция считает количество ресторанов доступных пользователю методом сравнение площади зоны доставки и суммы
    площадей треугольников на которые делит точка пользователя зону доставки
    user = [x, y] - координаты пользователя
    """     
    n=0
    for i in range(len(places)):
        # Сначала тест
        if sum_area_triangles(user, placetopoint(i))<sum_area_triangles(placetopoint(i)[0], placetopoint(i)[1:]): 
            print('Где-то ошибка')
        # Проверка по строгому равенству, но можно использовать и небольшой диапазон +-отклонение
        if sum_area_triangles(user, placetopoint(i))==sum_area_triangles(placetopoint(i)[0], placetopoint(i)[1:]):
            n+=1   
    return n

# Проверка
countplaces(usertopoint(0))

0

In [138]:
# Определяем сколько ресторанов доступно пользователю и записываем все в таблицу
df_count=pd.DataFrame()

for i in tnrange(len(users)):
    df_count.loc[i, 'id']=users[i]
    df_count.loc[i, 'number_of_places_available']=countplaces(usertopoint(i))

df_count.head()

A Jupyter Widget

,id,number_of_places_available
0,1.0,0.0
1,2.0,0.0
2,3.0,0.0


In [139]:
df_count=df_count.astype('int')
df_count.head()

,id,number_of_places_available
0,1,0
1,2,0
2,3,0


In [131]:
df_count.to_csv('number_of_places_available.csv')

In [140]:
# %debug